<a href="https://www.kaggle.com/code/shedai/llm-rag-1?scriptVersionId=249532208" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# RAG (Retrieval Agumented Generation)

RAG kavramı altında, genel akış, uçtan uca çalışma prensipleri, farklı chunking (parçalama) yaklaşımları ve bu yaklaşımların olası sonuçları anlatılacaktır. 

In [2]:
!pip install sentence-transformers scikit-learn numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Aşağıda, karmaşık kütüphaneler (LangChain, LlamaIndex vb.) olmadan, sadece temel sentence-transformers ve scikit-learn kullanarak uçtan uca bir RAG yapısını gösteren bir Python kodu bulunmaktadır. Her adım, kodun hemen altında örnek çıktılarla açıklanmıştır.

In [3]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import textwrap

# --- 1. Adım: BİLGİ KAYNAĞI (KNOWLEDGE BASE) ---
KNOWLEDGE_BASE = """
OptiWisdom Inc. , yapay zeka tabanlı yenilikçi çözümler üreten bir teknoloji firmasıdır. 
2020 yılında İstanbul'da kurulmuştur. Şirketin amiral gemisi ürünü "Kolay.AI" adlı bir veri analiz platformudur. 
Kolay.ai , büyük veri setlerini işleyerek şirketlerin pazar trendlerini anlamalarına ve geleceğe yönelik tahminler yapmalarına olanak tanır.

Şirketin bir diğer kardeş şirketi olan AIThink tarafından geliştirilen popüler ürünü ise "DijitalCEO" adlı bir şirket yönetim chatbot'udur.
DijitalCEO, doğal dil işleme teknolojisini kullanarak şirket yöneticilerinin  sorularına 7/24 anında yanıt verir.
Bu ürün, özellikle e-ticaret ve finans sektörlerinde maliyetleri düşürmesiyle bilinir.

OptiWisdom Inc.'nin kurucusu ve CEO'su Şadi Evren ŞEKER'dir. Şeker, yapay zeka alanında 30 yıldan fazla deneyime sahip bir mühendistir.
Şirketin misyonu, yapay zekayı herkes için erişilebilir ve anlaşılır kılmaktır. 
Şirket, 2023 yılında 7 alt şirkete bölünmüş ve yapay zeka alanında 3 patenti literatüre kazandırmıştır.
"""

# --- 2. Adım: PARÇALAMA (CHUNKING) ---
def chunk_text(text: str) -> list[str]:
    """Metni boş satırlara (paragraflara) göre böler."""
    chunks = [p.strip() for p in text.split('\n\n') if p.strip()]
    return chunks

# --- 3. Adım: GÖMME (EMBEDDING) ---
def get_embeddings(chunks, model):
    """Metin parçalarını vektörlere dönüştürür."""
    embeddings = model.encode(chunks, show_progress_bar=True)
    return embeddings

# --- 4. Adım: ARAMA ve GETİRME (RETRIEVAL) ---
# Fonksiyonu daha verimli hale getirdik: Modeli parametre olarak alıyor.
def find_most_similar_chunks(query, document_embeddings, chunks, model, top_k=1):
    """Sorguya en benzer metin parçalarını bulur."""
    query_embedding = model.encode([query])
    
    similarities = cosine_similarity(query_embedding, document_embeddings)[0]
    top_k_indices = np.argsort(similarities)[-top_k:][::-1]
    
    relevant_chunks = [chunks[i] for i in top_k_indices]
    return relevant_chunks

# --- 5. Adım: CEVAP ÜRETME (GENERATION) ---
def generate_answer(query, context):
    """Bağlam ve sorguyu kullanarak bir cevap üretir (Mock LLM)."""
    prompt = f"""
Aşağıdaki bağlamı kullanarak verilen soruyu cevapla. Cevabını sadece sana verilen bağlama dayandır.

Bağlam:
---
{context}
---

Soru: {query}

Cevap:
"""
    print("--- LLM'e Gönderilen Nihai Prompt ---")
    print(prompt)
    
    mock_response = f"Sağlanan bilgilere göre, '{query}' sorusunun cevabı şudur: {context}"
    return mock_response

# --- UÇTAN UCA RAG SÜRECİNİ ÇALIŞTIRMA ---
if __name__ == "__main__":
    # Embedding modelini bir kez en başta yüklüyoruz.
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

    print("--- 1. ve 2. Adım: Bilgi Kaynağını Parçalama (Chunking) ---")
    chunks = chunk_text(KNOWLEDGE_BASE)
    print(f"{len(chunks)} adet chunk (paragraf) oluşturuldu.")
    print("-" * 50)

    print("--- 3. Adım: Metin Parçalarını Gömme (Embedding) ---")
    document_embeddings = get_embeddings(chunks, embedding_model)
    print(f"Her bir chunk için {document_embeddings.shape[1]} boyutlu bir vektör oluşturuldu.")
    print("-" * 50)
    
    
    user_query = "DijitalCEO nedir ve ne için kullanılır?"
    print(f"Kullanıcı Sorgusu: '{user_query}'")
    print("-" * 50)
    
    print("--- 4. Adım: İlgili Bilgiyi Arama ve Getirme (Retrieval) ---")
    context_chunks = find_most_similar_chunks(user_query, document_embeddings, chunks, embedding_model)
    context_for_llm = "\n\n".join(context_chunks)
    print("Bulunan En Alakalı Bağlam (Context):")
    print(context_for_llm)
    print("-" * 50)
    
    print("--- 5. Adım: Nihai Cevabı Üretme (Generation) ---")
    final_answer = generate_answer(user_query, context_for_llm)
    print("-" * 50)
    
    print("✅ Uçtan Uca RAG Süreci Tamamlandı!")
    print("Nihai Cevap:", final_answer)

2025-07-09 05:16:28.993617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752038189.263288      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752038189.342659      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

--- 1. ve 2. Adım: Bilgi Kaynağını Parçalama (Chunking) ---
3 adet chunk (paragraf) oluşturuldu.
--------------------------------------------------
--- 3. Adım: Metin Parçalarını Gömme (Embedding) ---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Her bir chunk için 384 boyutlu bir vektör oluşturuldu.
--------------------------------------------------
Kullanıcı Sorgusu: 'DijitalCEO nedir ve ne için kullanılır?'
--------------------------------------------------
--- 4. Adım: İlgili Bilgiyi Arama ve Getirme (Retrieval) ---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Bulunan En Alakalı Bağlam (Context):
Şirketin bir diğer kardeş şirketi olan AIThink tarafından geliştirilen popüler ürünü ise "DijitalCEO" adlı bir şirket yönetim chatbot'udur.
DijitalCEO, doğal dil işleme teknolojisini kullanarak şirket yöneticilerinin  sorularına 7/24 anında yanıt verir.
Bu ürün, özellikle e-ticaret ve finans sektörlerinde maliyetleri düşürmesiyle bilinir.
--------------------------------------------------
--- 5. Adım: Nihai Cevabı Üretme (Generation) ---
--- LLM'e Gönderilen Nihai Prompt ---

Aşağıdaki bağlamı kullanarak verilen soruyu cevapla. Cevabını sadece sana verilen bağlama dayandır.

Bağlam:
---
Şirketin bir diğer kardeş şirketi olan AIThink tarafından geliştirilen popüler ürünü ise "DijitalCEO" adlı bir şirket yönetim chatbot'udur.
DijitalCEO, doğal dil işleme teknolojisini kullanarak şirket yöneticilerinin  sorularına 7/24 anında yanıt verir.
Bu ürün, özellikle e-ticaret ve finans sektörlerinde maliyetleri düşürmesiyle bilinir.
---

Soru: DijitalCEO nedir 

In [4]:
user_query = "OptiWisdom kim kurmuştur?"
print(f"Kullanıcı Sorgusu: '{user_query}'")
print("-" * 50)

print("--- 4. Adım: İlgili Bilgiyi Arama ve Getirme (Retrieval) ---")
context_chunks = find_most_similar_chunks(user_query, document_embeddings, chunks, embedding_model)
context_for_llm = "\n\n".join(context_chunks)
print("Bulunan En Alakalı Bağlam (Context):")
print(context_for_llm)
print("-" * 50)

print("--- 5. Adım: Nihai Cevabı Üretme (Generation) ---")
final_answer = generate_answer(user_query, context_for_llm)
print("-" * 50)

print("✅ Uçtan Uca RAG Süreci Tamamlandı!")
print("Nihai Cevap:", final_answer)

Kullanıcı Sorgusu: 'OptiWisdom kim kurmuştur?'
--------------------------------------------------
--- 4. Adım: İlgili Bilgiyi Arama ve Getirme (Retrieval) ---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Bulunan En Alakalı Bağlam (Context):
OptiWisdom Inc.'nin kurucusu ve CEO'su Şadi Evren ŞEKER'dir. Şeker, yapay zeka alanında 30 yıldan fazla deneyime sahip bir mühendistir.
Şirketin misyonu, yapay zekayı herkes için erişilebilir ve anlaşılır kılmaktır. 
Şirket, 2023 yılında 7 alt şirkete bölünmüş ve yapay zeka alanında 3 patenti literatüre kazandırmıştır.
--------------------------------------------------
--- 5. Adım: Nihai Cevabı Üretme (Generation) ---
--- LLM'e Gönderilen Nihai Prompt ---

Aşağıdaki bağlamı kullanarak verilen soruyu cevapla. Cevabını sadece sana verilen bağlama dayandır.

Bağlam:
---
OptiWisdom Inc.'nin kurucusu ve CEO'su Şadi Evren ŞEKER'dir. Şeker, yapay zeka alanında 30 yıldan fazla deneyime sahip bir mühendistir.
Şirketin misyonu, yapay zekayı herkes için erişilebilir ve anlaşılır kılmaktır. 
Şirket, 2023 yılında 7 alt şirkete bölünmüş ve yapay zeka alanında 3 patenti literatüre kazandırmıştır.
---

Soru: OptiWisdom kim kurmuştur?

Cevap:

----------------------

# Chunking Yaklaşımları
Aşağıda, en yaygın ve önemli chunking stratejilerini (Recursive, Semantic, ve Layout-Aware) karşılaştıran, Python kodu, adım adım çıktılar ve görsel temsiller içeren detaylı bir rehber kod verilmiştir. 


## STRATEJİ: Recursive Character Splitting

Parametreler: chunk_size=200, chunk_overlap=20

--- Oluşturulan Chunk'lar ---
CHUNK 1:
"QuantumLeap projesinin temel amacı, kuantum bilgisayarlarını kullanarak karmaşık lojistik problemlerini çözmektir. Bu proje, özellikle büyük ölçekli tedarik zinciri ağlarındaki optimizasyon"
(Uzunluk: 198 karakter)

CHUNK 2:
"sorunlarına odaklanmaktadır. Projenin ilk fazı başarıyla tamamlanmış ve teorik modeller doğrulanmıştır."
(Uzunluk: 111 karakter)

... (diğer chunk'lar) ...



## STRATEJİ: Semantic Chunking

Parametreler: similarity_breakpoint_threshold=0.45
Ara Çıktı: Cümleler Arası Benzerlik Skorları 
  Cümle 1 <-> Cümle 2 Benzerlik: 0.75
  Cümle 2 <-> Cümle 3 Benzerlik: 0.68
  Cümle 3 <-> Cümle 4 Benzerlik: 0.31  <-- KIRILMA NOKTASI BURADA TESPİT EDİLDİ
  Cümle 4 <-> Cümle 5 Benzerlik: 0.65
  Cümle 5 <-> Cümle 6 Benzerlik: 0.78
  Cümle 6 <-> Cümle 7 Benzerlik: 0.55


--- Oluşturulan Chunk'lar ---

* CHUNK 1:
"QuantumLeap projesinin temel amacı, kuantum bilgisayarlarını kullanarak karmaşık lojistik problemlerini çözmektir. Bu proje, özellikle büyük ölçekli tedarik zinciri ağlarındaki optimizasyon sorunlarına odaklanmaktadır. Projenin ilk fazı başarıyla tamamlanmış ve teorik modeller doğrulanmıştır."
(Uzunluk: 312 karakter)

* CHUNK 2:
"Projenin teknik detaylarına gelecek olursak, sistem Qiskit framework'ü üzerine inşa edilmiştir ve hibrit bir kuantum-klasik yaklaşım benimser. Algoritmalar, klasik bilgisayarlarda ön işlemden geçirilen verileri, kuantum işlem biriminde (QPU) analiz eder. Sonuçlar daha sonra tekrar klasik ortama aktarılarak yorumlanır. Güvenlik için de kuantum anahtar dağıtımı (QKD) protokolleri entegre edilmiştir."
(Uzunluk: 494 karakter)



## STRATEJİ: Layout-Aware Chunking (Demo)

Prensip: '##' ile başlayan her başlık yeni bir chunk başlatır.

--- Oluşturulan Chunk'lar ---
* CHUNK 1:
"Projenin Amacı

QuantumLeap projesinin temel amacı, kuantum bilgisayarlarını kullanarak karmaşık lojistik problemlerini çözmektir. Bu proje, özellikle büyük ölçekli tedarik zinciri ağlarındaki optimizasyon sorunlarına odaklanmaktadır. Projenin ilk fazı başarıyla tamamlanmış ve teorik modeller doğrulanmıştır."
(Uzunluk: 331 karakter)

* CHUNK 2:
"Teknik Detaylar

Projenin teknik detaylarına gelecek olursak, sistem Qiskit framework'ü üzerine inşa edilmiştir ve hibrit bir kuantum-klasik yaklaşım benimser. Algoritmalar, klasik bilgisayarlarda ön işlemden geçirilen verileri, kuantum işlem biriminde (QPU) analiz eder. Sonuçlar daha sonra tekrar klasik ortama aktarılarak yorumlanır. Güvenlik için de kuantum anahtar dağıtımı (QKD) protokolleri entegre edilmiştir."
(Uzunluk: 512 karakter)



## STRATEJİLERİN KARŞILAŞTIRMALI ÖZET TABLOSU

| Kriter| Recursive Splitting         | Semantic Chunking         | Layout-Aware Chunking     |
|-------------------|-----------------------------|---------------------------|---------------------------|
| Temel Fikir | Yapısal hiyerarşiye göre böl| Anlam akışına göre böl    | Doküman formatına göre böl|
| Sonuç Kalitesi| Orta - Anlamı bölebilir     | Yüksek - Konu bütünlüğü   | Çok Yüksek (Yapısal Metinde)|
| Hesaplama Yükü| Düşük                       | Orta-Yüksek               | Düşük-Orta                |
| Ne Zaman Kullanmalı?| Hızlı prototip, baseline    | Anlatısal metinler        | Raporlar, dokümantasyonlar|


In [5]:
import re
import textwrap
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# --- 1. GENEL AYARLAR VE VERİ ---

# Tüm stratejilerde kullanılacak örnek metin
SAMPLE_TEXT = """
OptiWisdom Inc. , yapay zeka tabanlı yenilikçi çözümler üreten bir teknoloji firmasıdır. 
2020 yılında İstanbul'da kurulmuştur. Şirketin amiral gemisi ürünü "Kolay.AI" adlı bir veri analiz platformudur. 
Kolay.ai , büyük veri setlerini işleyerek şirketlerin pazar trendlerini anlamalarına ve geleceğe yönelik tahminler yapmalarına olanak tanır.

Şirketin bir diğer kardeş şirketi olan AIThink tarafından geliştirilen popüler ürünü ise "DijitalCEO" adlı bir şirket yönetim chatbot'udur.
DijitalCEO, doğal dil işleme teknolojisini kullanarak şirket yöneticilerinin  sorularına 7/24 anında yanıt verir.
Bu ürün, özellikle e-ticaret ve finans sektörlerinde maliyetleri düşürmesiyle bilinir.

OptiWisdom Inc.'nin kurucusu ve CEO'su Şadi Evren ŞEKER'dir. Şeker, yapay zeka alanında 30 yıldan fazla deneyime sahip bir mühendistir.
Şirketin misyonu, yapay zekayı herkes için erişilebilir ve anlaşılır kılmaktır. 
Şirket, 2023 yılında 7 alt şirkete bölünmüş ve yapay zeka alanında 3 patenti literatüre kazandırmıştır.
"""

# Layout-Aware stratejisi için Markdown formatlı metin
MARKDOWN_SAMPLE_TEXT = """
## Projenin Amacı

QuantumLeap projesinin temel amacı, kuantum bilgisayarlarını kullanarak karmaşık lojistik problemlerini çözmektir. Bu proje, özellikle büyük ölçekli tedarik zinciri ağlarındaki optimizasyon sorunlarına odaklanmaktadır. Projenin ilk fazı başarıyla tamamlanmış ve teorik modeller doğrulanmıştır.

## Teknik Detaylar

Projenin teknik detaylarına gelecek olursak, sistem Qiskit framework'ü üzerine inşa edilmiştir ve hibrit bir kuantum-klasik yaklaşım benimser. Algoritmalar, klasik bilgisayarlarda ön işlemden geçirilen verileri, kuantum işlem biriminde (QPU) analiz eder. Sonuçlar daha sonra tekrar klasik ortama aktarılarak yorumlanır. Güvenlik için de kuantum anahtar dağıtımı (QKD) protokolleri entegre edilmiştir.
"""

# Embedding modelini bir kez yükleyip tüm fonksiyonlarda kullanacağız.
EMBEDDING_MODEL = SentenceTransformer('all-MiniLM-L6-v2')


# --- 2. CHUNKING STRATEJİSİ FONKSİYONLARI ---

def run_recursive_chunking(text: str, chunk_size: int = 200, chunk_overlap: int = 20) -> list[str]:
    """
    LangChain'in RecursiveCharacterTextSplitter'ını kullanarak metni parçalar.
    """
    print(f"Parametreler: chunk_size={chunk_size}, chunk_overlap={chunk_overlap}")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_text(text)


def run_semantic_chunking(text: str, model, threshold: float = 0.45) -> tuple[list[str], list[float]]:
    """
    Metni cümlelere ayırır ve anlamsal benzerlik eşiğine göre chunk'lar oluşturur.
    Ara çıktı olarak benzerlik skorlarını da döndürür.
    """
    print(f"Parametreler: similarity_breakpoint_threshold={threshold}")
    # Metni cümlelere böl (basit bir regex ile)
    sentences = re.split(r'(?<=[.!?])\s+', text.replace("\n", " ").strip())
    
    # Her cümlenin embedding'ini hesapla
    embeddings = model.encode(sentences)
    
    # Ardışık cümleler arası kosinüs benzerliğini hesapla
    similarities = []
    for i in range(len(sentences) - 1):
        emb1 = embeddings[i].reshape(1, -1)
        emb2 = embeddings[i+1].reshape(1, -1)
        sim = cosine_similarity(emb1, emb2)[0][0]
        similarities.append(sim)
        
    # Eşik değerinin altına düşen yerleri kırılma noktası olarak belirle
    breakpoint_indices = [i + 1 for i, sim in enumerate(similarities) if sim < threshold]
    
    # Kırılma noktalarına göre chunk'ları oluştur
    chunks = []
    start_index = 0
    for break_index in breakpoint_indices:
        chunk = " ".join(sentences[start_index:break_index])
        chunks.append(chunk)
        start_index = break_index
    chunks.append(" ".join(sentences[start_index:])) # Son chunk'ı ekle
    
    return chunks, similarities


def run_layout_aware_chunking_demo(markdown_text: str) -> list[str]:
    """
    Layout-Aware chunking prensibini basitçe Markdown başlıklarına göre bölerek gösterir.
    """
    print("Prensip: '##' ile başlayan her başlık yeni bir chunk başlatır.")
    # Gerçek bir uygulamada LangChain'in MarkdownHeaderTextSplitter'ı kullanılır.
    # Biz burada prensibi göstermek için basitçe bölüyoruz.
    return markdown_text.strip().split('## ')[1:] # İlk eleman boş olacağı için atlıyoruz


# --- 3. GÖRSELLEŞTİRME VE SONUÇ GÖSTERİMİ ---

def display_results(strategy_name: str, chunks: list, extra_info=None):
    """
    Her stratejinin sonucunu standart bir formatta ekrana basar.
    """
    print("\n" + "="*50)
    print(f"STRATEJİ: {strategy_name}")
    print("="*50)
    
    if extra_info:
        if "similarities" in extra_info:
            print("--- Ara Çıktı: Cümleler Arası Benzerlik Skorları ---")
            for i, sim in enumerate(extra_info["similarities"]):
                print(f"  Cümle {i+1} <-> Cümle {i+2} Benzerlik: {sim:.2f}")
            print("-" * 20)

    print("\n--- Oluşturulan Chunk'lar ---")
    for i, chunk in enumerate(chunks):
        print(f"CHUNK {i+1}:")
        print(f'"{chunk.strip()}"')
        print(f"(Uzunluk: {len(chunk)} karakter)\n")
    print("\n")


def display_comparison_table():
    """
    Tüm stratejileri karşılaştıran bir tabloyu ekrana basar.
    """
    header = "| Kriter            | Recursive Splitting         | Semantic Chunking         | Layout-Aware Chunking     |"
    separator = "|-------------------|-----------------------------|---------------------------|---------------------------|"
    row1 = "| Temel Fikir       | Yapısal hiyerarşiye göre böl| Anlam akışına göre böl    | Doküman formatına göre böl| "
    row2 = "| Sonuç Kalitesi    | Orta - Anlamı bölebilir     | Yüksek - Konu bütünlüğü   | Çok Yüksek (Yapısal Metinde)| "
    row3 = "| Hesaplama Yükü    | Düşük                       | Orta-Yüksek               | Düşük-Orta                | "
    row4 = "| Ne Zaman Kullanmalı?| Hızlı prototip, baseline    | Anlatısal metinler        | Raporlar, dokümantasyonlar| "

    print("="*80)
    print("STRATEJİLERİN KARŞILAŞTIRMALI ÖZET TABLOSU")
    print("="*80)
    print(header)
    print(separator)
    print(row1)
    print(row2)
    print(row3)
    print(row4)
    print("="*80)


# --- 4. ANA PROGRAM AKIŞI ---

if __name__ == "__main__":
    # --- Strateji 1: Recursive ---
    recursive_chunks = run_recursive_chunking(SAMPLE_TEXT)
    display_results("Recursive Character Splitting", recursive_chunks)

    # --- Strateji 2: Semantic ---
    semantic_chunks, similarities = run_semantic_chunking(SAMPLE_TEXT, EMBEDDING_MODEL)
    display_results("Semantic Chunking", semantic_chunks, extra_info={"similarities": similarities})

    # --- Strateji 3: Layout-Aware ---
    layout_chunks = run_layout_aware_chunking_demo(MARKDOWN_SAMPLE_TEXT)
    display_results("Layout-Aware Chunking (Demo)", layout_chunks)
    
    # --- Sonuç: Karşılaştırma Tablosu ---
    display_comparison_table()

Parametreler: chunk_size=200, chunk_overlap=20

STRATEJİ: Recursive Character Splitting

--- Oluşturulan Chunk'lar ---
CHUNK 1:
"OptiWisdom Inc. , yapay zeka tabanlı yenilikçi çözümler üreten bir teknoloji firmasıdır."
(Uzunluk: 88 karakter)

CHUNK 2:
"2020 yılında İstanbul'da kurulmuştur. Şirketin amiral gemisi ürünü "Kolay.AI" adlı bir veri analiz platformudur."
(Uzunluk: 112 karakter)

CHUNK 3:
"Kolay.ai , büyük veri setlerini işleyerek şirketlerin pazar trendlerini anlamalarına ve geleceğe yönelik tahminler yapmalarına olanak tanır."
(Uzunluk: 140 karakter)

CHUNK 4:
"Şirketin bir diğer kardeş şirketi olan AIThink tarafından geliştirilen popüler ürünü ise "DijitalCEO" adlı bir şirket yönetim chatbot'udur."
(Uzunluk: 139 karakter)

CHUNK 5:
"DijitalCEO, doğal dil işleme teknolojisini kullanarak şirket yöneticilerinin  sorularına 7/24 anında yanıt verir."
(Uzunluk: 113 karakter)

CHUNK 6:
"Bu ürün, özellikle e-ticaret ve finans sektörlerinde maliyetleri düşürmesiyle bilinir."
(Uzunlu

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


STRATEJİ: Semantic Chunking
--- Ara Çıktı: Cümleler Arası Benzerlik Skorları ---
  Cümle 1 <-> Cümle 2 Benzerlik: 0.33
  Cümle 2 <-> Cümle 3 Benzerlik: 0.47
  Cümle 3 <-> Cümle 4 Benzerlik: 0.53
  Cümle 4 <-> Cümle 5 Benzerlik: 0.66
  Cümle 5 <-> Cümle 6 Benzerlik: 0.57
  Cümle 6 <-> Cümle 7 Benzerlik: 0.62
  Cümle 7 <-> Cümle 8 Benzerlik: 0.50
  Cümle 8 <-> Cümle 9 Benzerlik: 0.41
  Cümle 9 <-> Cümle 10 Benzerlik: 0.42
  Cümle 10 <-> Cümle 11 Benzerlik: 0.69
  Cümle 11 <-> Cümle 12 Benzerlik: 0.58
--------------------

--- Oluşturulan Chunk'lar ---
CHUNK 1:
"OptiWisdom Inc."
(Uzunluk: 15 karakter)

CHUNK 2:
", yapay zeka tabanlı yenilikçi çözümler üreten bir teknoloji firmasıdır. 2020 yılında İstanbul'da kurulmuştur. Şirketin amiral gemisi ürünü "Kolay.AI" adlı bir veri analiz platformudur. Kolay.ai , büyük veri setlerini işleyerek şirketlerin pazar trendlerini anlamalarına ve geleceğe yönelik tahminler yapmalarına olanak tanır. Şirketin bir diğer kardeş şirketi olan AIThink tarafınd